# TEST KOLMOGOROV SMIRNOV

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Tue Sep 16 19:41:33 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,656924,35.1,1439402,76.9,1439402,76.9
Vcells,1224944,9.4,8388608,64.0,1924955,14.7


In [3]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

Loading required package: data.table

Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.


Attaching package: ‘R.utils’


The following object is masked from ‘package:utils’:

    timestamp


The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, isOpen, nullfile, parse, use, warnings




In [13]:
PARAM <- list()
PARAM$semilla_primigenia <- 102191

PARAM$experimento <- 1999
PARAM$dataset <- "analistajr_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [14]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("KS", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### 9.7.1   Preprocesamiento del dataset

#### 9.7.1.1  DT incorporar dataset

In [15]:
# lectura del dataset
dataset <- fread(paste0("/content/datasets/", PARAM$dataset))

In [16]:
View(dataset)


numero_de_cliente,foto_mes,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,⋯,Visa_mfinanciacion_limite,Visa_Fvencimiento,Visa_msaldototal,Visa_mlimitecompra,Visa_fultimo_cierre,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_mpagominimo,clase_ternaria
<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<chr>
29187730,201901,1,62,296,14301.42,43997.41,8970.82,-1262.22,5806.64,⋯,412896.5,-1337,127146.16,458783.6,22,5604,122057.93,50,11190.42,CONTINUA
29187961,201901,1,59,296,7542.75,34281.62,1960.95,-46.22,4804.79,⋯,412395.4,-1915,47726.51,458226.8,1,7604,48761.71,41,2087.94,CONTINUA
29193101,201901,1,66,349,5749.83,29663.24,4745.47,-3592.97,4449.20,⋯,207286.7,-1763,62541.14,230318.5,8,1540,26574.05,22,5841.54,CONTINUA
29193281,201901,1,52,142,12742.79,43614.97,12275.05,-718.19,1116.94,⋯,38005.2,-1154,17241.41,42228.0,1,2864,19809.29,8,1032.24,CONTINUA
29198891,201901,1,53,296,4237.90,23234.30,1176.26,-680.91,3291.01,⋯,50673.6,-211,22805.49,56304.0,1,6756,26750.83,4,1067.43,CONTINUA
29200770,201901,1,56,246,12527.87,60467.54,8240.80,281.61,3373.18,⋯,416459.8,-1763,8826.08,462742.9,8,4474,5590.22,5,1736.04,CONTINUA
29207790,201901,1,40,46,728.57,13014.84,219.22,-89.35,523.58,⋯,207286.7,-1549,3428.47,230318.5,8,1383,3079.74,7,304.98,CONTINUA
29207901,201901,0,81,296,4054.47,31669.65,1172.10,-131.76,2589.02,⋯,537595.4,-1154,2619.67,597328.2,1,7994,1506.68,2,0.00,CONTINUA
29215330,201901,1,48,237,3002.34,41679.79,920.64,-2332.47,4107.15,⋯,207286.7,-1215,8162.11,230318.5,8,7217,9100.89,7,480.93,CONTINUA


In [17]:
# Kolmogorov-Smirnov Test for Data Drift Detection
# Comparing each foto_mes distribution against 202109 as reference

# Load required libraries
require("data.table")

# Parameters - adjust as needed
PARAM <- list()
PARAM$dataset <- ""  # Will be set based on your dataset
PARAM$reference_month <- 202109  # Reference month for comparison
PARAM$alpha <- 0.05  # Significance level

# Check available foto_mes values
foto_mes_values <- sort(unique(dataset$foto_mes))
cat("Available foto_mes values:", paste(foto_mes_values, collapse = ", "), "\n")

# Verify reference month exists
if (!PARAM$reference_month %in% foto_mes_values) {
  stop("Reference month ", PARAM$reference_month, " not found in dataset!")
}

# Get reference data (202109)
reference_data <- dataset[foto_mes == PARAM$reference_month]
cat("Reference month", PARAM$reference_month, "has", nrow(reference_data), "observations\n")

# Get monetary columns for KS testing
# por como armé los nombres de campos,
# estos son los campos que expresan variables monetarias
campos_monetarios <- colnames(dataset)
campos_monetarios <- campos_monetarios[campos_monetarios %like% "^(m|Visa_m|Master_m|vm_m)"]

# Use monetary columns as our test variables
numeric_cols <- campos_monetarios

cat("Testing", length(numeric_cols), "monetary variables\n")
cat("Monetary fields found:", paste(campos_monetarios, collapse = ", "), "\n")
cat("Variables to test:", paste(head(numeric_cols, 10), collapse = ", "), 
    if(length(numeric_cols) > 10) "..." else "", "\n")

# Function to perform KS test for a single variable between two foto_mes
perform_ks_test <- function(var_name, month1_data, month2_data, month1_name, month2_name) {
  # Remove NA values
  var1 <- month1_data[[var_name]]
  var2 <- month2_data[[var_name]]
  
  var1 <- var1[!is.na(var1)]
  var2 <- var2[!is.na(var2)]
  
  if (length(var1) == 0 || length(var2) == 0) {
    return(list(
      variable = var_name,
      comparison = paste(month1_name, "vs", month2_name),
      n1 = length(var1),
      n2 = length(var2),
      ks_statistic = NA,
      p_value = NA,
      significant = NA,
      interpretation = "Insufficient data"
    ))
  }
  
  # Perform KS test
  ks_result <- ks.test(var1, var2)
  
  return(list(
    variable = var_name,
    comparison = paste(month1_name, "vs", month2_name),
    n1 = length(var1),
    n2 = length(var2),
    ks_statistic = round(ks_result$statistic, 4),
    p_value = round(ks_result$p.value, 3),
    significant = ks_result$p.value < PARAM$alpha,
    interpretation = ifelse(ks_result$p.value < PARAM$alpha, 
                           "Significant difference (data drift detected)", 
                           "No significant difference")
  ))
}

# Perform KS tests for each foto_mes against reference
all_results <- list()
result_count <- 0

cat("\n=== PERFORMING KOLMOGOROV-SMIRNOV TESTS ===\n")

# Test each foto_mes (except reference) against reference month
test_months <- setdiff(foto_mes_values, PARAM$reference_month)

for (test_month in test_months) {
  cat("\nTesting", test_month, "vs", PARAM$reference_month, "\n")
  
  test_data <- dataset[foto_mes == test_month]
  cat("  Sample size:", nrow(test_data), "observations\n")
  
  # Test each numeric variable
  month_results <- list()
  significant_vars <- 0
  
  for (var_name in numeric_cols) {
    result_count <- result_count + 1
    
    ks_result <- perform_ks_test(var_name, test_data, reference_data, 
                                test_month, PARAM$reference_month)
    
    month_results[[var_name]] <- ks_result
    all_results[[result_count]] <- ks_result
    
    if (!is.na(ks_result$significant) && ks_result$significant) {
      significant_vars <- significant_vars + 1
    }
  }
  
  cat("  Variables with significant drift:", significant_vars, "out of", length(numeric_cols), "\n")
}

# Convert results to data.table for analysis
results_dt <- rbindlist(all_results)

# Summary by foto_mes
cat("\n=== SUMMARY BY FOTO_MES ===\n")
summary_by_month <- results_dt[!is.na(significant), .(
  total_variables = .N,
  significant_drift = sum(significant),
  pct_drift = round(100 * sum(significant) / .N, 1),
  min_p_value = min(p_value, na.rm = TRUE),
  max_ks_stat = max(ks_statistic, na.rm = TRUE)
), by = .(foto_mes = sub(" vs.*", "", comparison))]

print(summary_by_month)

# Variables with most drift across months
cat("\n=== VARIABLES WITH MOST FREQUENT DRIFT ===\n")
drift_by_variable <- results_dt[!is.na(significant), .(
  months_tested = .N,
  months_with_drift = sum(significant),
  pct_months_drift = round(100 * sum(significant) / .N, 1),
  avg_ks_statistic = round(mean(ks_statistic, na.rm = TRUE), 4),
  min_p_value = min(p_value, na.rm = TRUE)
), by = variable][order(-pct_months_drift, -avg_ks_statistic)]

print(head(drift_by_variable, 20))

# Most significant cases (lowest p-values)
cat("\n=== MOST SIGNIFICANT DRIFT CASES ===\n")
most_significant <- results_dt[significant == TRUE][order(p_value)][1:20]

# Format p_value to always show 3 decimals
most_significant_formatted <- most_significant[, .(
  variable, 
  comparison, 
  ks_statistic, 
  p_value = sprintf("%.3f", p_value)
)]
print(most_significant_formatted)

# Export results
output_file <- "ks_test_results.csv"
fwrite(results_dt, output_file)
cat("\nDetailed results exported to:", output_file, "\n")

# Summary statistics
total_tests <- nrow(results_dt[!is.na(significant)])
significant_tests <- nrow(results_dt[significant == TRUE])

cat("\n=== OVERALL SUMMARY ===\n")
cat("Total KS tests performed:", total_tests, "\n")
cat("Tests showing significant drift:", significant_tests, "\n")
cat("Percentage of tests with drift:", round(100 * significant_tests / total_tests, 1), "%\n")
cat("Reference month:", PARAM$reference_month, "\n")
cat("Significance level:", PARAM$alpha, "\n")

cat("\nInterpretation:")
cat("\n- KS test detects differences in distributions between months")
cat("\n- Significant results (p <", PARAM$alpha, ") indicate data drift")
cat("\n- Higher KS statistics indicate larger distributional differences\n")




Available foto_mes values: 201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004, 202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108, 202109 
Reference month 202109 has 33094 observations
Testing 25 monetary variables
Monetary fields found: mrentabilidad, mrentabilidad_annual, mcomisiones, mactivos_margen, mpasivos_margen, mcuenta_corriente, mcaja_ahorro, mcuentas_saldo, mtarjeta_visa_consumo, mtarjeta_master_consumo, mprestamos_personales, mpayroll, mttarjeta_visa_debitos_automaticos, mcomisiones_mantenimiento, mtransferencias_recibidas, Master_mfinanciacion_limite, Master_msaldototal, Master_mlimitecompra, Master_mconsumototal, Master_mpagominimo, Visa_mfinanciacion_limite, Visa_msaldototal, Visa_mlimitecompra, Visa_mconsumototal, Visa_mpagominimo 
Variables to test: mrentabilidad, mrentabilidad_annual, mcomisiones, mactivos_margen, mpas

Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in th

  Variables with significant drift: 25 out of 25 

Testing 201902 vs 202109 
  Sample size: 24991 observations


Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in th

  Variables with significant drift: 25 out of 25 

Testing 201903 vs 202109 
  Sample size: 25097 observations


...

  Variables with significant drift: 25 out of 25 

Testing 202107 vs 202109 
  Sample size: 33053 observations


Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in th

  Variables with significant drift: 24 out of 25 

Testing 202108 vs 202109 
  Sample size: 33069 observations


Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in the presence of ties”
Warning message in ks.test.default(var1, var2):
“p-value will be approximate in th

  Variables with significant drift: 20 out of 25 

=== SUMMARY BY FOTO_MES ===
    foto_mes total_variables significant_drift pct_drift min_p_value
      <char>           <int>             <int>     <num>       <num>
 1:   201901              25                25       100           0
 2:   201902              25                25       100           0
 3:   201903              25                25       100           0
 4:   201904              25                25       100           0
 5:   201905              25                25       100           0
 6:   201906              25                25       100           0
 7:   201907              25                25       100           0
 8:   201908              25                25       100           0
 9:   201909              25                25       100           0
10:   201910              25                25       100           0
11:   201911              25                25       100           0
12:   201912            